In [135]:
import nltk
import pandas as pd
import numpy as np
import pymorphy2
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")

In [125]:
TARGET_WORD = 'язык'
POS = ['VERB', 'NUMR', 'ADJF', 'ADJS', 'PRTF', 'PRTS', 'ADVB', 'NOUN', 'NPRO', 'GRND', 'INFN', 'PRED']

Обучающая и тестовая коллекции для разных значений слова "язык"

In [119]:
train_lang = [
    'Чтобы книгу прочли на русском языке, ее нужно сначала перевести',
    'В отличе от остальной группы немецким языком он владел очень плохо',
    'Изучать иностранные языки нужно для общения с иностранцами',
    'А ведь у вас совсем другие цели — просто научиться общаться на иностранном языке, понимать его в разговоре и при чтении',
    'В индоиранских и греческом языках имеется большое число сходных слов',
    'На разных языках оно звучало по-разному',
    'В школах полезно детям читать отрывки на незнакомых языках, замечая, как понимается чуждая речь',
    'Если ты мечтаешь выучить китайский язык, тебе придётся найти на это время',
    'Не стоит учить иностранный язык только потому, что он вам понравился во время поездки за рубеж',
    'Получив образование за границей, он великолепно знал европейские языки'
]
train_tongue = [
    'Собака бежала,высунув язык',
    'язык участвует в процессе жевания',
    'Слизистая оболочка языка плотно сращена соединительной тканью',
    'Кошки, как и многие другие животные, активно используют язык для вылизывания себя и детёнышей',
    'Нужно держать свой язык за зубами и никто не пострадает',
    'В состоянии покоя язык имеет лопатообразную форму',
    'Верхушка языка прилежит к задней поверхности передних зубов',
    'Собаки с помощью языка осуществляют теплорегуляцию',
    'Основную массу языка составляют мышцы с их соединительнотканным аппаратом',
    'язык человека орган, помогающий диагностировать некоторые заболевания человека'
]
test_lang = [
    'я выучил русский язык очень быстро',
    'литературный язык очень красивый',
    'люди используют язык для достижения взаимопонимания',
    'человечество придумало язык жестов для людей, которые не могут использовать разговорный',
    'незнание нашего языка вяляется барьером для людей, говорящих на другом'
]

test_tongue = [
    'язык необходим при жевании пищи',
    'у собак очень длинный язык',
    'красный язык признак заболевания',
    'не все животные используют язык только для жевания пищи',
    'она показала язык и я все понял'
]

Функция, которая принимает на вход список предложений, возвращает список списков токенов каждого предложения в окрестности слова язык (2 слева и 2 справа)

In [120]:
def text_to_sub_sent(text, remove_special_words=True):
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    morph = pymorphy2.MorphAnalyzer()
    result = []
    for sent in text:
        tokens = tokenizer.tokenize(sent)
        if remove_special_words:
            tokens = [x for x in tokens if morph.parse(x)[0].tag.POS in POS]
        target_indices = [i for i, e in enumerate(tokens) if re.match('язык.*',e ) is not None]
        for i in range(len(target_indices)):
            if target_indices[i] == 0: 
                lower_bound = 0
                begin_none_cnt = 2
            elif target_indices[i] == 1: 
                lower_bound = 0
                begin_none_cnt = 1
            else:
                lower_bound = target_indices[i] - 2
                begin_none_cnt = 0
            if target_indices[i] == len(tokens) - 1:
                upper_bound = len(tokens) - 1
                end_none_cnt = 2
            elif target_indices[i] == len(tokens) - 2:
                upper_bound = len(tokens) - 1
                end_none_cnt = 1
            else:
                upper_bound = target_indices[i] + 2
                end_none_cnt = 0
            result.append([None]*begin_none_cnt + tokens[lower_bound:upper_bound + 1] + [None]*end_none_cnt)
    return result

Функция, которая возвращает обработанные данные функции text_to_sub_sent(), т.е. признаки

In [126]:
def get_features(subs, full_text, text, label, word_appearance=True, freq=True):
    if not word_appearance and not freq:
        raise ValueError
    result = np.empty((0, len(POS)*4)).astype(int)
    morph = pymorphy2.MorphAnalyzer()
    vectorizer = CountVectorizer()
    for sub in subs:  
        left_2 = pd.Series(data=np.zeros((len(POS))), index=POS)
        try:
            tag = morph.parse(sub[0])[0].tag.POS
            left_2[tag] = 1
        except AttributeError:
            pass      
        left_1 = pd.Series(data=np.zeros((len(POS))), index=POS)
        try:
            tag = morph.parse(sub[1])[0].tag.POS
            left_1[tag] = 1
        except AttributeError:
            pass     
        right_1 = pd.Series(data=np.zeros((len(POS))), index=POS)
        try:
            tag = morph.parse(sub[3])[0].tag.POS
            right_1[tag] = 1
        except AttributeError:
            pass   
        right_2 = pd.Series(data=np.zeros((len(POS))), index=POS)
        try:
            tag = morph.parse(sub[4])[0].tag.POS
            right_2[tag] = 1
        except AttributeError:
            pass    
        sub_features = np.array([pd.concat([left_2,left_1,right_1,right_2])]).astype(int)
        result = np.append(result, sub_features, axis=0)
    if word_appearance:
        vectorizer.fit_transform(full_text).toarray()
        result = np.hstack((result, vectorizer.transform(text).toarray()))
    if not freq:
        vectorizer.fit_transform(full_text).toarray()
        result = vectorizer.transform(text).toarray()
    return np.insert(result, result.shape[1], label, axis=1)

In [136]:
word_appearance=True
freq=True
model = MultinomialNB()
train = np.vstack((
    get_features(text_to_sub_sent(train_lang), train_lang + train_tongue, train_lang, 0, word_appearance, freq), 
    get_features(text_to_sub_sent(train_tongue), train_lang + train_tongue, train_tongue, 1, word_appearance, freq)
))
X = train[:,:-1]
y = train[:,-1:]
model.fit(X, y)


test = np.vstack((
    get_features(text_to_sub_sent(test_lang), train_lang + train_tongue, test_lang, 0, word_appearance, freq), 
    get_features(text_to_sub_sent(test_tongue), train_lang + train_tongue, test_tongue, 1, word_appearance, freq)
))
X_test = test[:,:-1]
y_test = test[:,-1:]
print('Точность классификатора: ')
accuracy_score(y_test,model.predict(X_test))

Точность классификатора: 


0.6

In [137]:
word_appearance=True
freq=False
model = MultinomialNB()
train = np.vstack((
    get_features(text_to_sub_sent(train_lang), train_lang + train_tongue, train_lang, 0, word_appearance, freq), 
    get_features(text_to_sub_sent(train_tongue), train_lang + train_tongue, train_tongue, 1, word_appearance, freq)
))
X = train[:,:-1]
y = train[:,-1:]
model.fit(X, y)


test = np.vstack((
    get_features(text_to_sub_sent(test_lang), train_lang + train_tongue, test_lang, 0, word_appearance, freq), 
    get_features(text_to_sub_sent(test_tongue), train_lang + train_tongue, test_tongue, 1, word_appearance, freq)
))
X_test = test[:,:-1]
y_test = test[:,-1:]
print('Точность классификатора: ')
accuracy_score(y_test,model.predict(X_test))

Точность классификатора: 


0.5

In [138]:
word_appearance=False
freq=True
model = MultinomialNB()
train = np.vstack((
    get_features(text_to_sub_sent(train_lang), train_lang + train_tongue, train_lang, 0, word_appearance, freq), 
    get_features(text_to_sub_sent(train_tongue), train_lang + train_tongue, train_tongue, 1, word_appearance, freq)
))
X = train[:,:-1]
y = train[:,-1:]
model.fit(X, y)


test = np.vstack((
    get_features(text_to_sub_sent(test_lang), train_lang + train_tongue, test_lang, 0, word_appearance, freq), 
    get_features(text_to_sub_sent(test_tongue), train_lang + train_tongue, test_tongue, 1, word_appearance, freq)
))
X_test = test[:,:-1]
y_test = test[:,-1:]
print('Точность классификатора: ')
accuracy_score(y_test,model.predict(X_test))

Точность классификатора: 


0.5

# Вывод

При тестировании установлено, что если использовать и части речи, и появление слова в предложении, точность составляет 0.6; если хотя бы один из наборов признаков не использовать, точность составляет 0.5